# Models Work

In [32]:
! git clone https://github.com/JulianKrese/project_chd/

Cloning into 'project_chd'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 64 (delta 17), reused 2 (delta 2), pack-reused 29
Receiving objects: 100% (64/64), 2.04 MiB | 7.76 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [36]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


# get training sets
X_train = pd.read_csv("/content/project_chd/X_train_cleaned.csv")
y_train = pd.read_csv("/content/project_chd/y_train_cleaned.csv")

# get testing sets
X_test = pd.read_csv("/content/project_chd/X_test_cleaned.csv")
y_test = pd.read_csv("/content/project_chd/y_test_cleaned.csv")


# trying linear regression with all variables
stock_reg = LinearRegression().fit(X_train, y_train)
print("R^2 = ", stock_reg.score(X_test, y_test))

# trying linear regression with min-max normalizing


# trying linear regression with choosing correlated variables


# trying linear regression with polynomial expanders (probably want to test multiple degrees)




column =  Unnamed: 0
type =  int64
False
----------------------
----------------------
column =  sex
type =  int64
False
----------------------
----------------------
column =  age
type =  int64
False
----------------------
----------------------
column =  currentSmoker
type =  int64
False
----------------------
----------------------
column =  cigsPerDay
type =  int64
False
----------------------
----------------------
column =  BPMeds
type =  int64
False
----------------------
----------------------
column =  prevalentStroke
type =  int64
False
----------------------
----------------------
column =  prevalentHyp
type =  int64
False
----------------------
----------------------
column =  diabetes
type =  int64
False
----------------------
----------------------
column =  totChol
type =  int64
False
----------------------
----------------------
column =  sysBP
type =  float64
False
----------------------
----------------------
column =  diaBP
type =  float64
False
---------------------

ValueError: Found input variables with inconsistent numbers of samples: [2744, 2756]